In [5]:
import numpy as np
import music21 as m21
from tqdm import tqdm

In [6]:
x = np.load('../data/nottingham/augmented_and_padded_data.npz')

In [7]:
names = x['name']
melody_pcps = x['melody_pcps']
chord_pcps = x['chord_pcps']
chord_roots = x['chord_roots']
chord_types = x['chord_types']

In [8]:
print(melody_pcps.shape)

(12252, 580, 12)


In [9]:
# key finding
def tonality_from_pcp( pcp ):
    major_profile = m21.analysis.discrete.KrumhanslSchmuckler().getWeights('major')
    minor_profile = m21.analysis.discrete.KrumhanslSchmuckler().getWeights('minor')
    major_corrs = np.zeros(12).astype(np.float32)
    minor_corrs = np.zeros(12).astype(np.float32)
    for i in range(12):
        major_corrs[i] = np.corrcoef( pcp, np.roll( 
            major_profile, i ) )[0][1]
        minor_corrs[i] = np.corrcoef( pcp, np.roll( 
            minor_profile, i ) )[0][1]
    major_max_idx = np.argmax( major_corrs )
    minor_max_idx = np.argmax( minor_corrs )
    major_max = np.max( major_corrs )
    minor_max = np.max( minor_corrs )
    if major_max > minor_max:
        return {'root': major_max_idx,
                'mode': 'major',
                'correlation': major_max}
    else:
        return {'root': minor_max_idx,
                'mode': 'minor',
                'correlation': minor_max}
# end tonality_from_pcp

In [10]:
# keep only c major pieces
c_melody_pcps = []
c_chord_pcps = []

for i in tqdm(range(melody_pcps.shape[0])):
    tm = tonality_from_pcp( np.sum(melody_pcps[i,:,:], axis=0) )
    tc = tonality_from_pcp( np.sum(chord_pcps[i,:,:], axis=0) )
    if tm['root'] == 0 and tm['mode'] == 'major' and tm['correlation'] > 0.8 and \
        tc['root'] == 0 and tc['mode'] == 'major' and tc['correlation'] > 0.8:
        c_melody_pcps.append( melody_pcps[i,:,:] )
        c_chord_pcps.append( chord_pcps[i,:,:] )

# print( tonality_from_pcp( np.sum(melody_pcps[0,:,:], axis=0) ) )
# print( tonality_from_pcp( np.sum(melody_pcps[1,:,:], axis=0) ) )
# print( tonality_from_pcp( np.sum(melody_pcps[2,:,:], axis=0) ) )
# print( np.sum(melody_pcps[0,:,:], axis=0) )

  0%|          | 0/12252 [00:00<?, ?it/s]

 13%|█▎        | 1615/12252 [00:05<00:39, 271.07it/s]/home/maximos/miniconda3/envs/torch/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/maximos/miniconda3/envs/torch/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
100%|██████████| 12252/12252 [00:45<00:00, 268.79it/s]


In [11]:
c_mel_np = np.array( c_melody_pcps )
print(c_mel_np.shape)
c_chr_np = np.array( c_chord_pcps )
print(c_chr_np.shape)

(526, 580, 12)
(526, 580, 12)


In [12]:
np.savez('../data/nottingham_c_major.npz', melody_pcps=c_mel_np, chord_pcps=c_chr_np)